In [1]:
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,aanybody,aapke,aaron,aarthi,abalance,abalanced,abarnacle,abassist,abelief,aberration,...,youwhy,youworried,youwould,youyou,yupi,zakir,zameen,zameer,zfg,zoom
indiatoday,1,0,0,0,1,2,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0
ndtv,0,0,0,2,0,0,0,0,0,0,...,1,1,0,2,0,1,0,2,0,0
republic,0,1,1,0,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,1,2


In [2]:
from gensim import matutils, models
import scipy.sparse

In [3]:
tdm = data.transpose()
tdm.head()

,indiatoday,ndtv,republic
aanybody,1,0,0
aapke,0,0,1
aaron,0,0,1
aarthi,0,2,0
abalance,1,0,1


In [5]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [6]:
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [7]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.008*"court" + 0.005*"want" + 0.005*"know" + 0.005*"say" + 0.005*"said" + 0.004*"im" + 0.004*"supreme" + 0.004*"come" + 0.004*"dont" + 0.004*"people"'),
 (1,
  '0.000*"know" + 0.000*"said" + 0.000*"say" + 0.000*"today" + 0.000*"supreme" + 0.000*"want" + 0.000*"people" + 0.000*"court" + 0.000*"im" + 0.000*"dont"')]

In [8]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.007*"court" + 0.004*"think" + 0.004*"act" + 0.004*"said" + 0.004*"given" + 0.003*"know" + 0.003*"come" + 0.003*"supreme" + 0.003*"today" + 0.003*"muslims"'),
 (1,
  '0.000*"court" + 0.000*"want" + 0.000*"im" + 0.000*"know" + 0.000*"say" + 0.000*"supreme" + 0.000*"people" + 0.000*"think" + 0.000*"today" + 0.000*"said"'),
 (2,
  '0.007*"court" + 0.006*"say" + 0.006*"want" + 0.006*"know" + 0.006*"im" + 0.005*"said" + 0.005*"dont" + 0.004*"supreme" + 0.004*"come" + 0.004*"people"')]

In [9]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.007*"court" + 0.006*"want" + 0.006*"say" + 0.005*"im" + 0.005*"said" + 0.005*"know" + 0.005*"today" + 0.004*"come" + 0.004*"supreme" + 0.004*"dont"'),
 (1,
  '0.000*"want" + 0.000*"court" + 0.000*"know" + 0.000*"said" + 0.000*"im" + 0.000*"say" + 0.000*"come" + 0.000*"supreme" + 0.000*"dont" + 0.000*"people"'),
 (2,
  '0.000*"court" + 0.000*"know" + 0.000*"want" + 0.000*"supreme" + 0.000*"said" + 0.000*"say" + 0.000*"come" + 0.000*"people" + 0.000*"im" + 0.000*"case"'),
 (3,
  '0.008*"court" + 0.006*"know" + 0.005*"case" + 0.005*"say" + 0.004*"said" + 0.004*"want" + 0.004*"okay" + 0.004*"party" + 0.003*"mean" + 0.003*"supreme"')]

This approach aint working out

## Nouns Only

In [22]:
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [23]:
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcript
indiatoday,the supreme courts our yoga verdict wasdeliver...
ndtv,hello and welcome youre watching leftright and...
republic,hello and welcome ladies and gentlementhis is ...


In [26]:
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
indiatoday,supreme verdict today judge unanimousverdict b...
ndtv,hello youre im father itis case dispute amatte...
republic,hello ladies gentlementhis edition sundaydebat...


In [28]:
from nltk.corpus import stopwords
add_stop_words = set(stopwords.words('english')) 

In [29]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

,aanybody,aapke,aaron,aarthi,abalance,abarnacle,abassist,abelief,aberration,ability,...,youso,youspread,youuninformative,youve,youwe,youyou,zakir,zameer,zfg,zoom
indiatoday,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
ndtv,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,1,2,1,2,0,0
republic,0,1,1,0,1,1,1,1,1,1,...,1,1,1,3,0,0,0,0,1,2


In [30]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [31]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.015*"court" + 0.006*"case" + 0.006*"people" + 0.005*"muslims" + 0.005*"land" + 0.005*"issue" + 0.004*"judgment" + 0.004*"today" + 0.004*"temple" + 0.004*"question"'),
 (1,
  '0.009*"court" + 0.007*"today" + 0.007*"country" + 0.006*"people" + 0.005*"im" + 0.005*"yesterday" + 0.004*"thing" + 0.004*"judgment" + 0.004*"verdict" + 0.004*"point"')]

In [32]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.013*"court" + 0.006*"act" + 0.006*"today" + 0.006*"judgment" + 0.006*"muslims" + 0.006*"people" + 0.005*"mosque" + 0.005*"question" + 0.005*"verdict" + 0.005*"evidence"'),
 (1,
  '0.014*"court" + 0.008*"people" + 0.007*"case" + 0.007*"today" + 0.006*"country" + 0.005*"im" + 0.005*"time" + 0.005*"point" + 0.004*"judgment" + 0.004*"party"'),
 (2,
  '0.001*"court" + 0.000*"today" + 0.000*"people" + 0.000*"verdict" + 0.000*"case" + 0.000*"country" + 0.000*"thing" + 0.000*"yesterday" + 0.000*"judgment" + 0.000*"im"')]

In [33]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.011*"court" + 0.008*"today" + 0.008*"country" + 0.008*"people" + 0.006*"im" + 0.006*"yesterday" + 0.005*"thing" + 0.005*"judgment" + 0.005*"verdict" + 0.005*"point"'),
 (1,
  '0.001*"court" + 0.000*"people" + 0.000*"case" + 0.000*"today" + 0.000*"im" + 0.000*"muslims" + 0.000*"verdict" + 0.000*"time" + 0.000*"day" + 0.000*"judgment"'),
 (2,
  '0.015*"court" + 0.007*"act" + 0.007*"today" + 0.007*"muslims" + 0.007*"judgment" + 0.006*"people" + 0.006*"mosque" + 0.005*"question" + 0.005*"verdict" + 0.005*"evidence"'),
 (3,
  '0.016*"court" + 0.010*"case" + 0.007*"party" + 0.006*"people" + 0.006*"temple" + 0.005*"land" + 0.004*"issue" + 0.004*"day" + 0.004*"bjp" + 0.004*"board"')]

## Nouns And Adjectives Both

In [35]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [36]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
indiatoday,supreme yoga verdict today judge unanimousverd...
ndtv,hello welcome youre leftright center im father...
republic,hello welcome ladies gentlementhis live editio...


In [37]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aanybody,aapke,aaron,aarthi,abalance,abalanced,abarnacle,abassist,abelief,aberration,...,youuninformative,youwant,youwe,youyou,yupi,zakir,zameen,zameer,zfg,zoom
indiatoday,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
ndtv,0,0,0,2,0,0,0,0,0,0,...,0,0,1,2,0,1,0,2,0,0
republic,0,1,1,0,1,0,1,1,1,1,...,1,1,0,0,0,0,0,0,1,2


In [38]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [39]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.003*"gandhi" + 0.002*"national" + 0.002*"media" + 0.002*"debate" + 0.002*"idea" + 0.002*"minute" + 0.002*"yeah" + 0.002*"lesson" + 0.002*"important" + 0.002*"kashi"'),
 (1,
  '0.004*"party" + 0.003*"board" + 0.002*"days" + 0.002*"relief" + 0.002*"mediation" + 0.002*"closure" + 0.002*"honorable" + 0.002*"stakeholders" + 0.002*"idea" + 0.002*"sunni"')]

In [42]:
# Let's start with 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.006*"party" + 0.004*"board" + 0.004*"days" + 0.003*"mediation" + 0.003*"honorable" + 0.003*"stakeholders" + 0.003*"sunni" + 0.002*"relief" + 0.002*"map" + 0.002*"arguments"'),
 (1,
  '0.003*"idea" + 0.002*"gandhi" + 0.002*"media" + 0.002*"national" + 0.002*"debate" + 0.002*"closure" + 0.002*"minute" + 0.002*"kashi" + 0.002*"yeah" + 0.002*"lesson"'),
 (2,
  '0.000*"party" + 0.000*"media" + 0.000*"debate" + 0.000*"gandhi" + 0.000*"idea" + 0.000*"important" + 0.000*"minute" + 0.000*"lesson" + 0.000*"yeah" + 0.000*"national"')]

In [41]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.003*"gandhi" + 0.003*"debate" + 0.003*"media" + 0.003*"national" + 0.003*"idea" + 0.003*"yeah" + 0.003*"minute" + 0.003*"lesson" + 0.002*"important" + 0.002*"vadra"'),
 (1,
  '0.007*"party" + 0.004*"board" + 0.004*"days" + 0.004*"mediation" + 0.004*"honorable" + 0.003*"stakeholders" + 0.003*"relief" + 0.003*"sunni" + 0.003*"map" + 0.002*"arguments"'),
 (2,
  '0.004*"closure" + 0.004*"idea" + 0.003*"mathura" + 0.003*"issues" + 0.002*"lives" + 0.002*"beginnings" + 0.002*"relief" + 0.002*"mp" + 0.002*"feeling" + 0.002*"community"'),
 (3,
  '0.000*"party" + 0.000*"board" + 0.000*"gandhi" + 0.000*"closure" + 0.000*"media" + 0.000*"idea" + 0.000*"days" + 0.000*"honorable" + 0.000*"national" + 0.000*"relief"')]

In [44]:
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.003*"closure" + 0.003*"idea" + 0.002*"issues" + 0.002*"mathura" + 0.002*"lives" + 0.002*"beginnings" + 0.001*"community" + 0.001*"relief" + 0.001*"feeling" + 0.001*"mp"'),
 (1,
  '0.004*"party" + 0.003*"gandhi" + 0.002*"national" + 0.002*"media" + 0.002*"debate" + 0.002*"yeah" + 0.002*"important" + 0.002*"board" + 0.002*"idea" + 0.002*"minute"')]

## Two Topics estimated

#### 1. Closure of issues, relief among community

#### 2. Party Politics and Media revolvind around the case

In [45]:
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(0, 'indiatoday'), (1, 'ndtv'), (1, 'republic')]

#### Topic 1 - india today
#### Topic 2 - ndtv and republic